01 - Generate JSON from Post-Inclusion/Exclusion CSV
- Enter path to CSV which has successful articles in the index column
- Enter path to JSON which has the articles with labeled sections

In [4]:
csv_path = "/Users/cu135/Library/CloudStorage/OneDrive-Personal/OneDrive_Documents/Work/Software/Research/nimlab/gpt_document_reader/ocr/keyword_matching_labeled_text/text_evaluations/text_evaluations/dropped_evaluations.csv"
json_path = "/Users/cu135/Library/CloudStorage/OneDrive-Personal/OneDrive_Documents/Work/Software/Research/nimlab/gpt_document_reader/ocr/keyword_matching_labeled_text/labeled_sections.json"

In [5]:
from calvin_utils.gpt_sys_review.json_utils import FilterPapers

# Initialize and run the FilterPapers class
filter_papers = FilterPapers(csv_path=csv_path, json_path=json_path)
filter_papers.run()

02 - Prepare Data Extraction Questions
- Dev Note: This is a repeat class and should be inherited from a .py

In [6]:
from calvin_utils.gpt_sys_review.gpt_utils import QuestionTemplate

question_template = QuestionTemplate()
question_template.data_extraction_questions()

Here are example data extraction questions:
{
    "Does the patient(s) represent(s) the whole experience of the investigator (center) or is the selection method unclear to the extent that other patients with similar presentation may not have been reported? (Good/Bad/Unclear)": "representative_case_quality",
    "Was patient\u2019s causal exposure clearly described? (Good/Bad/Unclear)": "causality_quality",
    "Were diagnostic tests or assessment methods and the results clearly described (amnesia tests)? (Good/Bad/Unclear)": "phenotyping_quality",
    "Were other alternative causes that may explain the observation (amnesia) ruled out? (Good/Bad/Unclear)": "workup_quality",
    "Were patient\u2019s demographics, medical history, comobidities clearly described? (Good/Bad/Unclear)": "clinical_covariates_quality",
    "Were patient\u2019s symptoms, interventions, and clinical outcomes clearly presented as a timeline? (Good/Bad/Unclear)": "history_quality",
    "Was the lesion image taken a

Copy the Template Dict from Above and Fill it Out As per the Example

In [7]:
question = {
    "Does the patient(s) represent(s) the whole experience of the investigator (center) or is the selection method unclear to the extent that other patients with similar presentation may not have been reported? (Good/Bad/Unclear)": "representative_case_quality",
    "Was patient\u2019s causal exposure clearly described? (Good/Bad/Unclear)": "causality_quality",
    "Were diagnostic tests or assessment methods and the results clearly described (amnesia tests)? (Good/Bad/Unclear)": "phenotyping_quality",
    "Were other alternative causes that may explain the observation (amnesia) ruled out? (Good/Bad/Unclear)": "workup_quality",
    "Were patient\u2019s demographics, medical history, comobidities clearly described? (Good/Bad/Unclear)": "clinical_covariates_quality",
    "Were patient\u2019s symptoms, interventions, and clinical outcomes clearly presented as a timeline? (Good/Bad/Unclear)": "history_quality",
    "Was the lesion image taken around the time of observation (amnesia) assessment? (Good/Bad/Unclear)": "temporal_causality_quality",
    "Is the case(s) described with sufficient details to allow other investigators to replicate the research or to allow practitioners make inferences related to their own practice? (Good/Bad/Unclear)": "history_quality_2"
}

03 - Extract Data with GPT


In [16]:
# Define file paths
api_key_path = "/Users/cu135/Library/CloudStorage/OneDrive-Personal/OneDrive_Documents/Work/Software/Research/nimlab/openai_key.txt"
json_file_path = "/Users/cu135/Library/CloudStorage/OneDrive-Personal/OneDrive_Documents/Work/Software/Research/nimlab/gpt_document_reader/ocr/keyword_matching_labeled_text/filtered_articles/filtered_labeled_sections.json"

# Define the keys you want to consider (exclude 'References')
keys_to_consider = ["Introduction", "Methods", "Results", "Discussion", "Conclusion"]  # Add or remove keys as per your requirement

# Define the type of article and questions
article_type = "research"

In [17]:
from calvin_utils.gpt_sys_review.gpt_utils import OpenAIChatEvaluator

In [18]:
evaluator = OpenAIChatEvaluator(api_key_path, json_file_path, keys_to_consider, article_type, question)
answers = evaluator.evaluate_all_files()
evaluator.save_to_json(answers)

100%|██████████| 1/1 [00:17<00:00, 17.02s/it]


04 - Convert results to a CSV

In [1]:
#Map responses to data
# Define a dictionary to map keywords to values
keyword_mapping = {
    -1: ["unclear", "unsure", "un"],
    0: ["poor", "bad", "negative"],
    1: ["good", "excellent", "positive"]
}


In [2]:
json_path = r'/Users/cu135/Library/CloudStorage/OneDrive-Personal/OneDrive_Documents/Work/Software/Research/nimlab/gpt_document_reader/ocr/keyword_matching_labeled_text/filtered_articles/text_evaluations/research_evaluations.json'

In [3]:
from calvin_utils.gpt_sys_review.json_utils import CustomSummarizer

# Create an instance of CustomSummarizer
custom_summarizer = CustomSummarizer(json_path=json_path)#, keyword_mapping=keyword_mapping)

# Run the custom summarization and save the results to CSV
result_df = custom_summarizer.run_custom()

# Display the resulting DataFrame
print(result_df)

/Users/cu135/.local/lib/python3.10/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


hi  {'chunk_1': 'Unclear. The text does not provide clear information on the selection method for the reported patient(s) and whether they represent the whole experience of the investigator (center).'}
hi  {'chunk_1': 'Unclear. The text does not clearly describe the causal exposure of the patient. It mentions that the clinical picture was initially attributed to an overdose of oxycodone, but it is not clear if this was the definitive cause or if there were other factors involved.'}
hi  {'chunk_1': "Good. The text clearly describes the diagnostic tests and assessment methods used to evaluate the patient's amnesia. It specifically mentions the Wechsler Memory Scale-Revised, Benton Visual Retention Test, and the Rey-Osterreith Complex Figure Test. The results of these tests are also provided, indicating the extent of impairment in different areas of memory."}
hi  {'chunk_1': 'Unclear. The text does not explicitly mention whether other alternative causes that may explain the observation of